In [52]:
from datetime import datetime, timedelta
import xlrd

import pandas

In [54]:
xlsx_path = '/Users/shniu/Desktop/business_records.xlsx'

workbook = xlrd.open_workbook(xlsx_path)
for sheet in workbook.sheets():
    print(sheet.name)

客户类 - 商务活动记录——客户类
投资类 - 商务活动记录——投资类
学术类 - 商务活动记录——学术类
其他交流类 - 商务活动记录——其他交流类
好东西 - 商务活动记录——其他交流类


In [81]:
def caculate_end_date(note_index, s_date, s_index=5):
    diff_index = note_index - s_index
    start_datetime = datetime.strptime(s_date, '%Y-%m-%d')
    end_datetime = start_datetime + timedelta(days=diff_index)
    
    return end_datetime.strftime('%Y.%m.%d')


def split_records_by_name(raw_sheet, org_index=2, s_date=None, s_index=5):
    new_sheets = {}
    
    for note_rows_cols, note in raw_sheet.cell_note_map.items():
        org_name = raw_sheet.cell_value(note_rows_cols[0], org_index)
        updated_on = caculate_end_date(note_rows_cols[1], s_date, s_index=s_index)
        
        note_text = note.text
        note_text = note_text.replace('limeng qian:\n', '')
#         note_text = note_text.replace('\n', '')
        
        tmp_df = pd.DataFrame(data=[[updated_on, note_text, note_rows_cols[1]]],
                              columns=['date', 'content', 'colx'])
        
        if org_name not in new_sheets:
            new_sheets[org_name] = tmp_df
        else:
            new_sheets[org_name] = pd.concat([new_sheets[org_name], tmp_df], ignore_index=True)
            
    return new_sheets


def save(sheets, file_name):
    print(file_name)
    writer = pd.ExcelWriter(file_name)
    
    for org_name, df in sheets.items():
        df = df.sort_values(by='colx')
        df[['date', 'content']].to_excel(writer, org_name, index=False)
        
    writer.save()

In [82]:
configs = {
    'common': {
        'start_index':  5,
        'start_date': '2017-07-24',
        'org_index': 2
    },
    '投资类 - 商务活动记录——投资类': {
        'start_index':  3,
        'start_date': '2017-07-24',
        'org_index': 1
    },
    '学术类 - 商务活动记录——学术类': {
        'start_index':  3,
        'start_date': '2017-07-24',
        'org_index': 1
    },
    '其他交流类 - 商务活动记录——其他交流类': {
        'start_index':  3,
        'start_date': '2017-07-24',
        'org_index': 1
    },
    '好东西 - 商务活动记录——其他交流类': {
        'start_index':  3,
        'start_date': '2017-07-24',
        'org_index': 1
    },
}

for sheet in workbook.sheets():
    config = configs.get(sheet.name, configs['common'])
    file_name = '%s.xlsx' % sheet.name.replace(' ', '')
    save(split_records_by_name(sheet, 
                               org_index=config['org_index'], 
                               s_date=config['start_date'],
                              s_index=config['start_index']), 
         file_name)

客户类-商务活动记录——客户类.xlsx
投资类-商务活动记录——投资类.xlsx
学术类-商务活动记录——学术类.xlsx
其他交流类-商务活动记录——其他交流类.xlsx
好东西-商务活动记录——其他交流类.xlsx
